In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [7]:
df_nba = pd.read_csv("./Dataset/games.csv")
df_nba.columns = df_nba.columns.str.replace("_"," ")
print(df_nba.dtypes)
display(df_nba)

GAME DATE EST        object
GAME ID               int64
GAME STATUS TEXT     object
HOME TEAM ID          int64
VISITOR TEAM ID       int64
SEASON                int64
TEAM ID home          int64
PTS home            float64
FG PCT home         float64
FT PCT home         float64
FG3 PCT home        float64
AST home            float64
REB home            float64
TEAM ID away          int64
PTS away            float64
FG PCT away         float64
FT PCT away         float64
FG3 PCT away        float64
AST away            float64
REB away            float64
HOME TEAM WINS        int64
dtype: object


,GAME DATE EST,GAME ID,GAME STATUS TEXT,HOME TEAM ID,VISITOR TEAM ID,SEASON,TEAM ID home,PTS home,FG PCT home,FT PCT home,...,AST home,REB home,TEAM ID away,PTS away,FG PCT away,FT PCT away,FG3 PCT away,AST away,REB away,HOME TEAM WINS
0,22/12/2022,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,22/12/2022,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,21/12/2022,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,21/12/2022,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,21/12/2022,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,6/10/2014,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
26647,6/10/2014,11400004,Final,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,...,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
26648,6/10/2014,11400005,Final,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,...,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
26649,5/10/2014,11400002,Final,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,...,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1
